In [40]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [60]:
#Mcdonalds Big Mac Index
Mcdf = pd.read_json("big-mac-index-by-state-2024.json")  # Read JSON file directly


In [13]:
#Minimum Wage data
#excel to dataframe
minwagedf = pd.read_excel('statistic_id238997_us-minimum-wage-2024-by-state.xlsx')

In [61]:
#median/ average income
states=[]
averages= []
medians= []

url = "https://www.sofi.com/learn/content/average-salary-in-us/"
r = requests.get(url)
text = r.text
soup= BeautifulSoup(r.text)
trs = soup.find_all('tr')

for row in trs:  
    cols = row.find_all("td")  # Get all columns in the row
    if len(cols) >= 3:  # Ensure it has enough columns
        states.append(cols[0].text.strip())
        averages.append(int(cols[1].text.replace("$", "").replace(",", "")))
        medians.append(int(cols[2].text.replace("$", "").replace(",", "")))
medavgdf = pd.DataFrame({'State':states, 'Average':averages, 'Median':medians})

In [42]:
#starbucks data
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.dailymail.co.uk/yourmoney/consumer/article-13320199/starbucks-coffee-prices-us.html")
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Now try finding the rows
rows = soup.find_all(class_=['datawrapper-xp8Z2-1c5djey', 'svelte-1smti70'])
print(rows)

# url = "https://www.dailymail.co.uk/yourmoney/consumer/article-13320199/starbucks-coffee-prices-us.html"
# r = requests.get(url)
# text = r.text
# soup= BeautifulSoup(r.text)
# # tr_tags = soup.find_all(class_='datawrapper-xp8Z2-1c5djey')
# # tr_tags
# rows = soup.find_all(class_=['datawrapper-xp8Z2-1c5djey', 'svelte-1smti70'])
# rows
# print(soup.prettify())

[]


In [34]:
# titles = soup.find_all('titles', {'class':'PromoImageOnTopCircular-subtitle'})
# divs = soup.find_all('div', {'class':'PromoImageOnTopCircular-description'})

# names = []
# links = []
# for h3 in h3_tags:
#     name = h3.get_text().strip()
#     link = h3.find('a')['href']
#     names.append(name)
#     links.append(link)

# titles = []
# for div in divs:
#     title = div.get_text().strip()
#     titles.append(title)

# df = pd.DataFrame({'name':names, 'Titles':titles, 'links':links})

# websites = []
# emails = []
# offices = []    
# phones = []
# for link in df['links']:
#     r_sub = requests.get(link)
#     soup_sub = BeautifulSoup(r_sub.text)
#     try:
#         website = soup_sub.find('div', {'class':"EmployeePage-websiteLink"}).get_text().strip()
#         websites.append(website)
#     except:
#         websites.append('None')
#     try:
#         email = soup_sub.find('div', {'class':"EmployeePage-email"}).get_text().strip()
#         emails.append(email)
#     except:
#         emails.append('None')
#     try:
#         phone = soup_sub.find('div', {'class':"EmployeePage-phoneNumber"}).get_text().strip()
#         phones.append(phone)
#     except:
#         phones.append('None')
#     try:
#         office = soup_sub.find('div', {'class':"EmployeePage-address"}).get_text().strip()
#         offices.append(office)
#     except:
#         offices.append('None')

# new_data = pd.DataFrame({'links':links, 'websites':websites, 'emails':emails, 'phones':phones, 'offices':offices})
# df = pd.concat([df, new_data], axis=1)

# df =df.drop('links', axis=1)
# #df


In [64]:
#cost of living
livingcostdf = pd.read_json("cost-of-living-index-by-state-2025.json")  # Read JSON file directly
livingcostdf = livingcostdf.drop(columns=['stateFlagCode'])
livingcostdf

,state,CostOfLivingIndex_CostOfLivingIndex_num_2024,CostOfLivingIndex_GroceriesCostsIndex_num_2024,CostOfLivingIndex_HousingCostsIndex_num_2024,CostOfLivingIndex_UtilitiesCostsIndex_num_2024,CostOfLivingIndex_TransportationCostsIndex_num_2024,CostOfLivingIndex_HealthCostsIndex_num_2024,CostOfLivingIndex_MiscCostsIndex_num_2024
0,Hawaii,186.9,130.4,310.0,198.0,133.3,121.1,130.5
1,Massachusetts,145.9,104.4,218.8,150.0,109.5,125.7,115.3
2,California,144.8,111.7,208.7,139.8,136.7,107.7,115.3
3,District of Columbia,141.9,105.8,222.4,102.6,107.7,117.4,113.3
4,Alaska,123.8,127.1,118.0,152.8,114.3,150.0,118.9
5,New York,123.3,103.9,168.0,99.7,106.8,108.9,106.9
6,Maryland,115.3,107.1,135.8,110.3,101.7,100.9,108.9
7,New Jersey,114.6,104.2,136.9,100.6,105.6,107.5,107.5
8,Vermont,114.4,106.2,129.7,112.6,95.8,111.4,111.1
9,Washington,114.2,105.6,126.9,92.6,123.7,118.1,109.9
